ResNet（Residual Network）是深度学习领域具有里程碑意义的卷积神经网络（CNN），其核心创新在于**残差结构（Residual Block）**，该结构有效解决了深层网络训练中的梯度消失/爆炸问题，使得训练超深层网络（如1000层以上）成为可能。以下从残差结构和ResNet整体两方面详细介绍：


### 一、残差结构（Residual Block）
残差结构是ResNet的核心组件，其设计初衷是解决“深层网络性能退化”问题——当网络深度增加到一定程度后，模型精度会先提升再饱和，最终甚至下降（并非过拟合，而是优化困难）。


#### 1. 核心思想：学习残差而非直接映射
传统深层网络中，每一层试图学习输入到输出的“直接映射”（记为$H(x)$）；而残差结构则通过**跳跃连接（Skip Connection）**，将输入$x$直接传到输出，转而学习“残差函数”$F(x) = H(x) - x$。此时，网络的实际输出为：  
$$H(x) = F(x) + x$$  

- 若$H(x)$是理想的映射，残差$F(x)$可能更接近0，网络更容易学习（只需拟合小扰动）；  
- 梯度反向传播时，残差$F(x)$的梯度可直接通过跳跃连接传递到浅层，避免梯度消失。  


#### 2. 两种典型残差块结构
根据网络深度不同，残差块分为两种类型：


##### （1）基本块（Basic Block）
用于较浅的ResNet（如ResNet-18、34），结构简单，由**2个3x3卷积层**堆叠而成，配合批量归一化（BN）和ReLU激活函数。  
- 流程：输入$x$ → 卷积层1（3x3，BN+ReLU） → 卷积层2（3x3，BN） → 与跳跃连接的$x$相加 → ReLU输出。  
- 示意图简化：  
  $$x \rightarrow [Conv3x3 \rightarrow BN \rightarrow ReLU \rightarrow Conv3x3 \rightarrow BN] + x \rightarrow ReLU$$  


##### （2）瓶颈块（Bottleneck Block）
用于较深的ResNet（如ResNet-50、101、152），通过1x1卷积减少计算量（“瓶颈”得名于中间层通道数更少）。结构为**1x1卷积 → 3x3卷积 → 1x1卷积**，同样配合BN和ReLU。  
- 作用：1x1卷积先将输入通道数“降维”（减少计算），3x3卷积提取特征，最后1x1卷积“升维”恢复通道数。  
- 流程：输入$x$ → 1x1卷积（降维，BN+ReLU） → 3x3卷积（特征提取，BN+ReLU） → 1x1卷积（升维，BN） → 与跳跃连接的$x$相加 → ReLU输出。  


#### 3. 跳跃连接（Skip Connection）的细节
跳跃连接是残差结构的关键，其作用是将输入$x$“短路”到输出，实现$F(x) + x$的操作。根据输入$x$与主路径输出的维度是否一致，跳跃连接分为两种：  
- **恒等捷径（Identity Shortcut）**：当主路径输出与$x$的通道数、空间尺寸（高/宽）相同时，直接将$x$与主路径输出相加（无需额外操作）。  
- **投影捷径（Projection Shortcut）**：当主路径因降采样（步长=2）或通道数变化导致维度不一致时，通过1x1卷积对$x$进行“维度调整”（匹配主路径输出的通道数和尺寸），再相加。  


### 二、ResNet整体结构
ResNet通过堆叠上述残差块构建深层网络，其核心是“以残差块为单元”，逐步提升网络深度和特征提取能力。


#### 1. 核心特点
- 解决深层网络训练难题：通过残差结构的跳跃连接，梯度可直接从深层回流到浅层，避免梯度消失/爆炸，使得训练1000层以上的网络成为可能。  
- 性能随深度提升：突破了“深度饱和”限制，更深的网络（如ResNet-152）比浅层网络（如ResNet-18）能学习更复杂的特征，精度更高。  


#### 2. 典型版本及结构差异
ResNet有多个经典版本（ResNet-18/34/50/101/152），命名中的数字代表“网络总层数”（含卷积层和全连接层），核心差异在于**残差块的类型和数量**：  

| 版本       | 残差块类型 | 残差块数量（按组划分） | 总层数 |
|------------|------------|------------------------|--------|
| ResNet-18  | 基本块     | 2+2+2+2                | 18     |
| ResNet-34  | 基本块     | 3+4+6+3                | 34     |
| ResNet-50  | 瓶颈块     | 3+4+6+3                | 50     |
| ResNet-101 | 瓶颈块     | 3+4+23+3               | 101    |
| ResNet-152 | 瓶颈块     | 3+8+36+3               | 152    |


#### 3. 通用结构框架
所有ResNet的整体流程一致，可分为5个部分：  
1. **初始卷积层**：7x7大卷积（步长2）+ 3x3最大池化（步长2），快速缩小特征图尺寸（如输入224x224→56x56），并提取低级特征。  
2. **残差块组**：4组堆叠的残差块（对应上表中的“按组划分”），每组内通过残差块逐步提升特征图的通道数（如64→128→256→512），并通过步长=2的卷积实现2次降采样（最终特征图尺寸从56x56→28x28→14x14→7x7）。  
3. **全局平均池化**：将最后一个残差块的输出（如7x7x512）通过全局平均池化压缩为1x1x512的向量。  
4. **全连接层**：将向量映射到任务类别（如ImageNet的1000类），输出分类概率。  


#### 4. 经典ResNet-50示例
以应用最广泛的ResNet-50为例，具体结构如下：  
- 初始层：7x7卷积（64通道，步长2）→ BN → ReLU → 3x3池化（步长2）→ 输出尺寸56x56x64。  
- 残差块组1（3个瓶颈块）：保持尺寸56x56，通道数从64→256（通过投影捷径调整初始$x$的维度）。  
- 残差块组2（4个瓶颈块）：步长=2降采样至28x28，通道数256→512。  
- 残差块组3（6个瓶颈块）：步长=2降采样至14x14，通道数512→1024。  
- 残差块组4（3个瓶颈块）：步长=2降采样至7x7，通道数1024→2048。  
- 最终层：全局平均池化（7x7→1x1）→ 全连接层（1000类）。  


### 三、总结
- **残差结构**是ResNet的核心，通过“学习残差+跳跃连接”解决了深层网络的优化难题，使得梯度可高效反向传播。  
- **ResNet**通过堆叠残差块实现了超深层网络的训练，突破了传统CNN的深度限制，极大推动了计算机视觉（如分类、检测、分割）的发展。后续许多网络（如ResNeXt、DenseNet）均借鉴了残差连接的思想。

# 残差块儿无池化层 

是的，ResNet中的残差块（无论是基本块还是瓶颈块）**本身不包含池化层**。残差块的核心是通过卷积层学习残差函数，然后通过跳跃连接将输入与输出相加。不过，ResNet网络中确实存在下采样操作，但这些操作是通过**带步长的卷积层**而非传统的池化层实现的。以下是详细说明：


## 一、残差块内部无池化层
残差块的设计目标是学习输入与输出之间的残差映射，其结构主要由**卷积层、批量归一化（BN）和ReLU激活函数**组成。以两种典型残差块为例：
1. **基本块（Basic Block）**：  
   由两个3x3卷积层（每个卷积层后接BN和ReLU）组成，没有池化层。  
   流程：`x → Conv3x3 → BN → ReLU → Conv3x3 → BN → [+x] → ReLU`

2. **瓶颈块（Bottleneck Block）**：  
   由1x1（降维）、3x3、1x1（升维）三个卷积层组成，同样不含池化层。  
   流程：`x → Conv1x1 → BN → ReLU → Conv3x3 → BN → ReLU → Conv1x1 → BN → [+x] → ReLU`


## 二、ResNet中的下采样如何实现？
虽然残差块内部没有池化层，但ResNet网络在某些位置需要进行**下采样**（如特征图尺寸从56x56→28x28）。ResNet通过以下方式实现下采样：
1. **带步长的卷积层**：  
   在残差块的第一个卷积层（通常是3x3卷积或1x1卷积）中设置**步长=2**，从而实现特征图尺寸减半。例如，ResNet-50的瓶颈块组2-4的第一个1x1卷积使用步长=2进行下采样。

2. **投影捷径（Projection Shortcut）**：  
   当下采样导致输入$x$与主路径输出维度不一致时，跳跃连接需要通过1x1卷积对$x$进行投影，使其维度与主路径匹配。此时，1x1卷积也会使用步长=2，确保$x$的空间尺寸与主路径输出一致。


## 三、ResNet中的池化层位置
ResNet中**仅在网络前端和后端存在池化层**，而非残差块内部：
1. **初始层的最大池化**：  
   在第一个卷积层（7x7卷积）之后，使用3x3最大池化（步长=2）对输入图像进行下采样，这是网络的第一个下采样操作。

2. **全局平均池化（GAP）**：  
   在所有残差块组之后，使用全局平均池化将特征图压缩为1x1向量，用于后续的全连接层分类。


## 四、为什么不使用池化层？
1. **保留更多信息**：  
   池化层（尤其是最大池化）会丢失部分空间信息，而带步长的卷积层可以在降采样的同时保留更多特征信息。

2. **梯度流动更顺畅**：  
   残差块的设计依赖于跳跃连接传递梯度，避免使用池化层可以减少梯度路径中的非线性操作，进一步缓解梯度消失问题。


## 五、总结
- **残差块内部**确实没有池化层，其结构以卷积层为主。  
- **ResNet网络中的下采样**通过带步长的卷积层实现，而非传统池化层。  
- **池化层仅存在于网络的初始阶段（最大池化）和末尾（全局平均池化）**。

这种设计使ResNet能够在保持特征表达能力的同时，高效地进行深层网络训练。

## 代码实现  

In [3]:
import torch.nn as nn 
import torch.nn.functional as F

In [7]:
class ResnetbasicBlock(nn.Module): 
    def __init__(self,in_channels,out_channels): 
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels
                              ,out_channels, 
                               kernel_size=3, #用3*3的卷积核
                               padding=1, 
                               bias=False
                              )
        #设置bn层
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels
                              ,out_channels, 
                               kernel_size=3, #用3*3的卷积核
                               padding=1, 
                               bias=False
                              )
        #设置bn层
        self.bn2 = nn.BatchNorm2d(out_channels)
    def forward(self,x): #x写为input也可
        residual = x
        x = F.relu(self.bn1(self.conv1(x)),inplace=True) #inplace = True为就地改变
        x = F.relu(self.bn2(self.conv2v(x)),inplace=True) 

        #关键的一步,实现F(x)+x 
        x+=residual 
        return F.relu(x)
        
        

In [10]:
import torchvision
model = torchvision.models.resnet18(weights=None)

In [11]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
model = torchvision.models.resnet18??

Signature:
torchvision.models.resnet18(
    *,
    weights: Optional[torchvision.models.resnet.ResNet18_Weights] = None,
    progress: bool = True,
    **kwargs: Any,
) -> torchvision.models.resnet.ResNet
Source:   
@register_model()
@handle_legacy_interface(weights=("pretrained", ResNet18_Weights.IMAGENET1K_V1))
def resnet18(*, weights: Optional[ResNet18_Weights] = None, progress: bool = True, **kwargs: Any) -> ResNet:
    """ResNet-18 from `Deep Residual Learning for Image Recognition <https://arxiv.org/abs/1512.03385>`__.

    Args:
        weights (:class:`~torchvision.models.ResNet18_Weights`, optional): The
            pretrained weights to use. See
            :class:`~torchvision.models.ResNet18_Weights` below for
            more details, and possible values. By default, no pre-trained
            weights are used.
        progress (bool, optional): If True, displays a progress bar of the
            download to stderr. Default is True.
        **kwargs: parameters passed to t